In [11]:
# Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import pandas as pd

In [12]:
# Set up splinter browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA Mars News

In [3]:
# URL of page to be scraped
mars_news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(mars_news_url)

# Parse the html
mars_news_html = browser.html
mars_news_soup = bs(mars_news_html, "html.parser")

In [4]:
# Grab the latest mars news title and text
news_title = mars_news_soup.find('div', class_='content_title').text
news_p = mars_news_soup.find('div', class_='article_teaser_body').text
print(news_title)
print(news_p)

InSight Is the Newest Mars Weather Service
By collecting data around the clock, NASA's lander will provide unique science about the Martian surface.


# JPL Mars Space Images - Featured Image

In [5]:
# URL of page to be scraped
mars_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(mars_image_url)

# Parse the html
mars_image_html = browser.html
mars_image_soup = bs(mars_image_html, "html.parser")

In [6]:
# Grab the featured mars image
featured_image = mars_image_soup.find('article')['style'].replace("background-image: url('","").replace("');","")
#print(featured_image)

# Store the main mars image url
main_mars_image_url = "https://www.jpl.nasa.gov"

# Combine the main + featured image urls
featured_image_url = main_mars_image_url + featured_image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19101-1920x1200.jpg



# Mars Weather

In [7]:
# URL of page to be scraped
mars_weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(mars_weather_url)

# Parse the html
mars_weather_html = browser.html
mars_weather_soup = bs(mars_weather_html, "html.parser")

In [8]:
# Grab the latest mars weather tweet
mars_tweet = mars_weather_soup.find_all('div', class_='js-tweet-text-container')
#print(mars_tweet)

# Select only the tweets with weather information
for tweet in mars_tweet:
    mars_weather = tweet.find('p').text
    if "sol" in mars_weather:
        print(mars_weather)
        break

InSight sol 81 (2019-02-17), high -17/2F, low -95/-138F, pressure at 7.23hPa, winds from the WNW at 12 mph gusting to 37.8 mph

Welcome to the Mars Weather team @NASAInSight!
https://mars.nasa.gov/insight/weather/ …pic.twitter.com/SH12FvcMfv


# Mars Facts

In [9]:
# URL of page to be scraped
mars_facts_url = "https://space-facts.com/mars/"

# Read the html using pandas
mars_table = pd.read_html(mars_facts_url)
#print(type(mars_table))

# Convert the mars table into a DataFrame 
mars_df = mars_table[0]

# Fix the table to make it more presentable
mars_df.columns = ['Measurements','Records']
mars_df.set_index('Measurements', inplace=True)
mars_df

,Records
Measurements,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


# Mars Hemispheres

In [13]:
# URL of page to be scraped
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_hemisphere_url)

# Parse the html
mars_hemisphere_html = browser.html
mars_hemisphere_soup = bs(mars_hemisphere_html, "html.parser")

In [39]:
# Store the main mars hemisphere url
main_mars_hemisphere_url = 'https://astrogeology.usgs.gov'

# Grab the featured mars image
hemisphere_image_urls = []
mars_hemisphere = mars_hemisphere_soup.find_all('div', class_='item')
for hemisphere in mars_hemisphere:
    
    # Collecting the mars hemisphere titles
    mars_hemisphere_title = hemisphere.find('h3').text
    #print(mars_hemisphere_title)
    
    # Collect the mars hemisphere img urls
    mars_hemisphere_url = hemisphere.find('a', class_='itemLink product-item')['href']
    #print(mars_hemisphere_url)
    mars_hemisphere_full_url = main_mars_hemisphere_url + mars_hemisphere_url
    #print(mars_hemisphere_full_url)
    
    # Visit each individual hemisphere image url
    browser.visit(mars_hemisphere_full_url)
    mars_hemisphere_img_html = browser.html
    mars_hemisphere_img_soup = bs(mars_hemisphere_img_html, 'html.parser')
    
    # Collect the full mars hemisphere img urls
    mars_hemisphere_part_img_url = mars_hemisphere_img_soup.find('img', class_='wide-image')['src']
    #print(mars_hemisphere_img)
    mars_hemisphere_img_url = main_mars_hemisphere_url + mars_hemisphere_part_img_url
    #print(mars_hemisphere_img_url)
    
    # Create a title + url dictionary and append to variable hemisphere image url list
    hemisphere_image_urls.append({"title":mars_hemisphere_title, "img_url":mars_hemisphere_img_url})
hemisphere_image_urls
    

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]